Markdown Cell
* https://jupyter.brynmawr.edu/services/public/dblank/CS371%20Cognitive%20Science/2016-Fall/Programming%20a%20Chess%20Player.ipynb
* https://python-chess.readthedocs.io/en/latest/

In [2]:
"""
Python cell
"""
print("hello world")

hello world


In [9]:

import chess
board = chess.Board()
board.turn
board.turn = chess.WHITE
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [17]:
board.piece_at(chess.C1)

list(board.legal_moves)
move = list(board.legal_moves)[0]
move.uci()
board.san(move)

'Nh3'

In [20]:
import random

def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

random_player(board)

for i in range(10):
    print(random_player(board))

d2d4
e2e3
g2g4
g2g4
e2e4
f2f3
g1f3
g2g4
e2e4
a2a3


In [24]:
import time
from IPython.display import display, HTML, clear_output

def who(player):
    return "White" if player == chess.WHITE else "Black"

def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg()
    else:
        return "<pre>" + str(board) + "</pre>"

def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

play_game(random_player, random_player)

AttributeError: 'Board' object has no attribute '_repr_svg'

In [26]:
def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

play_game(random_player, human_player)

AttributeError: 'Board' object has no attribute '_repr_svg'

In [27]:
counts = {None: 0, True: 0, False: 0}
for i in range(10):
    result, msg, board = play_game(random_player, random_player, visual=None)
    counts[result] += 1
    print(counts)
counts

{None: 1, True: 0, False: 0}
{None: 1, True: 0, False: 1}
{None: 2, True: 0, False: 1}
{None: 2, True: 0, False: 2}
{None: 2, True: 1, False: 2}
{None: 3, True: 1, False: 2}
{None: 4, True: 1, False: 2}
{None: 5, True: 1, False: 2}
{None: 6, True: 1, False: 2}
{None: 7, True: 1, False: 2}


{None: 7, True: 1, False: 2}

In [32]:
def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        move.score = staticAnalysis(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse = True)
    return moves[0].uci()

board = chess.Board()
board.pieces(chess.ROOK, True)

list(board.pieces(chess.ROOK, True))
len(board.pieces(chess.ROOK, True))

def staticAnalysis(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

    play_game(player1, random_player)

In [35]:
def pawnPromotionAnalysis(board, move, my_color):
    score = random.random()

    board.push(move)

    promotion_bonus = 50  

    back_rank = chess.BB_RANK_8 if my_color == chess.WHITE else chess.BB_RANK_1
    for square in chess.SquareSet(back_rank):
        piece = board.piece_at(square)
        if piece and piece.color == my_color and piece.piece_type == chess.QUEEN:
            score += promotion_bonus

    return score
    play_game(player1, random_player)